# Assignment 2.1

In [1]:
import requests,pprint,json,datetime,time
from gzip import decompress
from json import loads
from requests import get
import pandas as pd
from pymongo import MongoClient
from secrets import *

In [2]:
def getData(city,countrycode):
    requestString='https://api.openweathermap.org/data/2.5/weather?q=%s,%s&APPID=%s' % (city,countrycode,key)
    
    res=requests.get(requestString)
    
    return res

In [7]:
getData('Toronto', 'CA')

NameError: name 'key' is not defined

In [4]:
res.status_code

NameError: name 'res' is not defined

In [5]:
pprint.pprint(res.json())

NameError: name 'res' is not defined

# Ingesting data into MongoDB

## Create DB and collection

In [6]:
client=MongoClient('localhost',27017)

In [7]:
weatherCollectionCanada=client.weatherdb.weather

weatherCollection.drop()

## Get list of cities

In [8]:
df = pd.read_json(decompress(get("http://bulk.openweathermap.org/sample/city.list.json.gz").content))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209579 entries, 0 to 209578
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       209579 non-null  int64 
 1   name     209579 non-null  object
 2   state    209579 non-null  object
 3   country  209579 non-null  object
 4   coord    209579 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.0+ MB


In [10]:
df.head()

,id,name,state,country,coord
0,833,Ḩeşār-e Sefīd,,IR,"{'lon': 47.159401, 'lat': 34.330502}"
1,2960,‘Ayn Ḩalāqīm,,SY,"{'lon': 36.321911, 'lat': 34.940079}"
2,3245,Taglag,,IR,"{'lon': 44.98333, 'lat': 38.450001}"
3,3530,Qabāghlū,,IR,"{'lon': 46.168499, 'lat': 36.173302}"
4,5174,‘Arīqah,,SY,"{'lon': 36.48336, 'lat': 32.889809}"


## Limit to Canadian cities

In [11]:
df_canada = df[df['country']=='CA']

In [12]:
df_canada.head()

,id,name,state,country,coord
161742,5881688,Fort St. James,,CA,"{'lon': -124.262749, 'lat': 54.455971}"
161743,5881774,Abbey,,CA,"{'lon': -108.751427, 'lat': 50.733391}"
161744,5881791,Abbotsford,,CA,"{'lon': -122.252571, 'lat': 49.05798}"
161745,5881792,Abbotsford,,CA,"{'lon': -122.285873, 'lat': 49.049679}"
161746,5881868,Aberdeen,,CA,"{'lon': -106.284477, 'lat': 52.31678}"


In [13]:
df_canada[df['name']=='Vancouver']

C:\Users\Cyrus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,id,name,state,country,coord
164583,6173331,Vancouver,,CA,"{'lon': -123.119339, 'lat': 49.24966}"


In [14]:
len(df_canada)

3302

# Cycle through cities and Query MongoDB for useful information

In [15]:
canadian_cities_list = list(df_canada['name'])

In [16]:
pprint.pprint(canadian_cities_list)

['Fort St. James',
 'Abbey',
 'Abbotsford',
 'Abbotsford',
 'Aberdeen',
 'Abitibi-Témiscamingue',
 'Aboushagan Road',
 'Acadia Valley',
 'Acme',
 'Acton',
 'Acton Vale',
 'Aden',
 'Admiral',
 'Agassiz',
 'Agincourt',
 'Aguanish',
 'Airdrie',
 'Ajax',
 'Aklavik',
 'Akulivik',
 'Alameda',
 'Albanel',
 'Albany',
 'Albatross',
 'Alberni',
 'Alberta',
 'Alberton',
 'Alberton South',
 'Albertville',
 'Albrights Corner',
 'Albury',
 'Alderson',
 'Alert Bay',
 'Alexander',
 'Alexandria',
 'Alexis Creek',
 'Alfred',
 'Algonquin Park',
 'Alida',
 'Alix',
 'Allan',
 'Allenford',
 'Alliance',
 'Alliston',
 'Alma',
 'Almdals Cove',
 'Alsask',
 'Altona',
 'Alvena',
 'Amaranth',
 'Amherst',
 'Amherst',
 'Amherstburg',
 'Amherstburg',
 'Amos',
 'Amyot',
 'Andrew',
 'Angliers',
 'Angus',
 'Annapolis Royal',
 'Antigonish',
 'Applegrove',
 'Apsley',
 'Arborfield',
 'Arborg',
 'Arcadia',
 'Arcola',
 'Arctic Bay',
 'Ardmore',
 'Ardrossan',
 'Arelee',
 'Argyle',
 'Argyle Shore',
 'Armley',
 'Armour Heights'

In [18]:
canadian_cities_list_short = canadian_cities_list[1:100]

In [19]:
for city in canadian_cities_list_short:
    res = getData(city,'ca')
    weatherCollectionCanada.insert_one(res.json())

In [20]:
for data in weatherCollectionCanada.find():
    pprint.pprint(data)

{'_id': ObjectId('5ef832c869c478fe66b943cf'),
 'base': 'stations',
 'clouds': {'all': 97},
 'cod': 200,
 'coord': {'lat': 50.73, 'lon': -108.75},
 'dt': 1593320805,
 'id': 5881774,
 'main': {'feels_like': 288.72,
          'humidity': 85,
          'pressure': 992,
          'temp': 288.71,
          'temp_max': 288.71,
          'temp_min': 288.71},
 'name': 'Abbey',
 'sys': {'country': 'CA',
         'id': 2032470,
         'sunrise': 1593255853,
         'sunset': 1593315116,
         'type': 3},
 'timezone': -21600,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 77, 'speed': 1.34}}
{'_id': ObjectId('5ef832c969c478fe66b943d0'),
 'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 49.06, 'lon': -122.25},
 'dt': 1593320805,
 'id': 5881791,
 'main': {'feels_like': 283.79,
          'humidity': 93,
          'pressure': 1011,
          'temp': 285.82,
        

In [21]:
for data in weatherCollectionCanada.find({},{ "_id": 0, "name": 1, "coord.lon":1, "coord.lat":1, "weather.main":1, 
                                             "weather.description":1, "main.temp":1, "main.humidity":1 }):
    pprint.pprint(data)

{'coord': {'lat': 50.73, 'lon': -108.75},
 'main': {'humidity': 85, 'temp': 288.71},
 'name': 'Abbey',
 'weather': [{'description': 'overcast clouds', 'main': 'Clouds'}]}
{'coord': {'lat': 49.06, 'lon': -122.25},
 'main': {'humidity': 93, 'temp': 285.82},
 'name': 'Abbotsford',
 'weather': [{'description': 'light intensity shower rain', 'main': 'Rain'}]}
{'coord': {'lat': 49.06, 'lon': -122.25},
 'main': {'humidity': 93, 'temp': 285.82},
 'name': 'Abbotsford',
 'weather': [{'description': 'light intensity shower rain', 'main': 'Rain'}]}
{'coord': {'lat': 52.32, 'lon': -106.28},
 'main': {'humidity': 87, 'temp': 289.04},
 'name': 'Aberdeen',
 'weather': [{'description': 'broken clouds', 'main': 'Clouds'}]}
{'coord': {'lat': 48.1, 'lon': -77.78},
 'main': {'humidity': 93, 'temp': 291.15},
 'name': 'Abitibi-Témiscamingue',
 'weather': [{'description': 'overcast clouds', 'main': 'Clouds'}]}
{'coord': {'lat': 45.98, 'lon': -64.35},
 'main': {'humidity': 87, 'temp': 288.15},
 'name': 'Aboush

In [22]:
cur = weatherCollectionCanada.find({},{ "_id": 0, "name": 1, "coord.lon":1, "coord.lat":1, "weather.main":1, 
                                             "weather.description":1, "main.temp":1, "main.humidity":1 })

In [23]:
a = len(canadian_cities_list_short)

In [24]:
# make an empty list
WeatherData = [[] for _ in range(a)]
i = 0
header_list = ['City','Longitude','Latitude','WeatherType','WeatherDescription','Temperature','Humidity']

for data in cur:
    WeatherData[i].append(data['name'])
    WeatherData[i].append(data['coord']['lon'])
    WeatherData[i].append(data['coord']['lat'])
    WeatherData[i].append(data['weather'][0]['main'])
    WeatherData[i].append(data['weather'][0]['description'])
    WeatherData[i].append(data['main']['temp'])
    WeatherData[i].append(data['main']['humidity'])
    i = i + 1

In [25]:
pd.DataFrame(WeatherData, columns=header_list)

,City,Longitude,Latitude,WeatherType,WeatherDescription,Temperature,Humidity
0,Abbey,-108.75,50.73,Clouds,overcast clouds,288.71,85
1,Abbotsford,-122.25,49.06,Rain,light intensity shower rain,285.82,93
2,Abbotsford,-122.25,49.06,Rain,light intensity shower rain,285.82,93
3,Aberdeen,-106.28,52.32,Clouds,broken clouds,289.04,87
4,Abitibi-Témiscamingue,-77.78,48.10,Clouds,overcast clouds,291.15,93
...,...,...,...,...,...,...,...
94,Aupaluk,-69.60,59.30,Clear,clear sky,277.14,74
95,Aurora,-79.47,44.00,Clear,clear sky,292.39,88
96,Austin,-98.94,49.95,Clear,clear sky,296.55,75
97,Authier,-78.85,48.73,Clouds,few clouds,288.30,100


## Find the max recorded temperatures per station

In [26]:
cur = weatherCollectionCanada.find({},{ "_id": 0, "name": 1, "main.temp":1, "weather.main":1, "weather.description":1 })
for data in cur.sort('main.temp',-1):
    pprint.pprint(data)

{'main': {'temp': 298.15},
 'name': 'Almdals Cove',
 'weather': [{'description': 'clear sky', 'main': 'Clear'}]}
{'main': {'temp': 296.92},
 'name': 'Altona',
 'weather': [{'description': 'clear sky', 'main': 'Clear'}]}
{'main': {'temp': 296.57},
 'name': 'Amaranth',
 'weather': [{'description': 'few clouds', 'main': 'Clouds'}]}
{'main': {'temp': 296.55},
 'name': 'Austin',
 'weather': [{'description': 'clear sky', 'main': 'Clear'}]}
{'main': {'temp': 296.23},
 'name': 'Ashern',
 'weather': [{'description': 'light rain', 'main': 'Rain'}]}
{'main': {'temp': 295.43},
 'name': 'Arborg',
 'weather': [{'description': 'clear sky', 'main': 'Clear'}]}
{'main': {'temp': 295.15},
 'name': 'Ashville',
 'weather': [{'description': 'broken clouds', 'main': 'Clouds'}]}
{'main': {'temp': 294.49},
 'name': 'Amherstburg',
 'weather': [{'description': 'broken clouds', 'main': 'Clouds'}]}
{'main': {'temp': 294.49},
 'name': 'Amherstburg',
 'weather': [{'description': 'broken clouds', 'main': 'Clouds'}]}


In [27]:
# Clear
cur = weatherCollectionCanada.find({"weather.main":{'$eq':'Clear'}},{ "_id": 0, "name": 1, "main.temp":1, "weather.main":1, "weather.description":1 })
count = 0
for data in cur:
    count = count + 1

print(count)

17


In [28]:
# Clouds
cur = weatherCollectionCanada.find({"weather.main":{'$eq':'Clouds'}},{ "_id": 0, "name": 1, "main.temp":1, "weather.main":1, "weather.description":1 })
count = 0
for data in cur:
    count = count + 1

print(count)

57


In [29]:
# Fog
cur = weatherCollectionCanada.find({"weather.main":{'$eq':'Fog'}},{ "_id": 0, "name": 1, "main.temp":1, "weather.main":1, "weather.description":1 })
count = 0
for data in cur:
    count = count + 1

print(count)

1


In [30]:
# Mist
cur = weatherCollectionCanada.find({"weather.main":{'$eq':'Mist'}},{ "_id": 0, "name": 1, "main.temp":1, "weather.main":1, "weather.description":1 })
count = 0
for data in cur:
    count = count + 1

print(count)

2


In [31]:
# Rain
cur = weatherCollectionCanada.find({"weather.main":{'$eq':'Rain'}},{ "_id": 0, "name": 1, "main.temp":1, "weather.main":1, "weather.description":1 })
count = 0
for data in cur:
    count = count + 1

print(count)

22


## Get the datetime of temperature per station

In [35]:
pipeline = []

# Filter out dirty rows
pipeline.append({'$match':{'name':{'$exists':True}}})

# Sort by name (esc) and temperature (asc)
pipeline.append({'$sort':{'name':1,'main.temp':-1}})

# Group by name, grab maximum temperature and date of temperature
# Sorted by temperature so grab first document from each group
pipeline.append({'$group':{'_id':'$name','Temp':{'$first':'$main.temp'},'date':{'$first':'$dt'}}})

# Limit to first 50 cities
pipeline.append({'$limit':50})

In [36]:
cur = weatherCollectionCanada.aggregate(pipeline=pipeline)

In [37]:
def getTimestamp(dt):
    return pd.dataframe.fromtimestamp(dt)

In [38]:
def getDate(tstamp):
    dt = datetime.datetime.fromtimestamp(tstamp)
    return dt.strftime('%b %d - %H:%m')

In [47]:
for d in cur:
    print (getDate(d['date']),d['_id'],d['Temp'])
    print ('')

Jun 27 - 21:06 Alsask 289.15

Jun 27 - 21:06 Airdrie 281.86

Jun 27 - 21:06 Alexander 294.15

Jun 27 - 21:06 Alfred 290.76

Jun 27 - 21:06 Argyle Shore 292.04

Jun 27 - 21:06 Arviat 275.15

Jun 27 - 21:06 Ascot Corner 291.15

Jun 27 - 21:06 Alert Bay 286.15

Jun 27 - 21:06 Armstrong Station 290.15

Jun 27 - 21:06 Asinkaanumevatt 293.49

Jun 27 - 21:06 Alberni 286.48

Jun 27 - 21:06 Alberton South 289.9

Jun 27 - 21:06 Alix 285.15

Jun 27 - 21:06 Aklavik 286.15

Jun 27 - 21:06 Amherst 288.15

Jun 27 - 21:06 Atnarko 285.95

Jun 27 - 21:06 Arnprior 290.58

Jun 27 - 21:06 Ajax 292.77

Jun 27 - 21:06 Albertville 287.15

Jun 27 - 21:06 Amherstburg 294.49

Jun 27 - 21:06 Arcadia 284.73

Jun 27 - 21:06 Ardmore 287.15

Jun 27 - 21:06 Alexis Creek 282.68

Jun 27 - 21:06 Abbotsford 285.82

Jun 27 - 21:06 Apsley 292.07

Jun 27 - 21:06 Acme 283.16

Jun 27 - 21:06 Arctic Bay 280.3

Jun 27 - 21:06 Antigonish 289.98

Jun 27 - 21:06 Albrights Corner 290.35

Jun 27 - 21:06 Asbestos 291.21

Jun 27 - 21:0